In [1]:
from datasets import load_dataset, DatasetDict

dataset = DatasetDict()

# Laden des Datasets von der bereinigten CSV-Datei
dataset["train"] = load_dataset("audiofolder", data_dir="/Users/peterkompiel/python_scripts/asr4memory/processing_files/whisper-train/_output/eg085_01_01/", split="train")
dataset["test"] = load_dataset("audiofolder", data_dir="/Users/peterkompiel/python_scripts/asr4memory/processing_files/whisper-train/_output/eg085_01_01/", split="test")

# Überprüfen des ersten Datensatzes
print(dataset['train'][0])
print(dataset['test'][0])

/opt/miniconda3/envs/whisper-train/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'audio': {'path': '/Users/peterkompiel/python_scripts/asr4memory/processing_files/whisper-train/_output/eg085_01_01/data/train/eg085_01_01_audio_segment_1.wav', 'array': array([-0.00045776, -0.00012207, -0.00035095, ...,  0.0035553 ,
        0.0027771 ,  0.00285339]), 'sampling_rate': 16000}, 'transcription': 'Ich kriegte die Einberufung zum Arbeitsdienst.'}
{'audio': {'path': '/Users/peterkompiel/python_scripts/asr4memory/processing_files/whisper-train/_output/eg085_01_01/data/test/eg085_01_01_audio_segment_1.wav', 'array': array([-0.00726318, -0.00692749, -0.00590515, ...,  0.00479126,
        0.00495911,  0.00480652]), 'sampling_rate': 16000}, 'transcription': 'Da wurden wir als ganz kleine, halbe Mediziner ausgebildet.'}


In [2]:
# Gesamtes DatasetDict anzeigen
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 1324
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 331
    })
})


In [3]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [4]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Polish", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
input_str = dataset["train"][0]["transcription"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Input:                 Ich kriegte die Einberufung zum Arbeitsdienst.
Decoded w/ special:    <|startoftranscript|><|pl|><|transcribe|><|notimestamps|>Ich kriegte die Einberufung zum Arbeitsdienst.<|endoftext|>
Decoded w/out special: Ich kriegte die Einberufung zum Arbeitsdienst.
Are equal:             True


In [6]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Polish", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
print(dataset["train"][0])

{'audio': {'path': '/Users/peterkompiel/python_scripts/asr4memory/processing_files/whisper-train/_output/eg085_01_01/data/train/eg085_01_01_audio_segment_1.wav', 'array': array([-0.00045776, -0.00012207, -0.00035095, ...,  0.0035553 ,
        0.0027771 ,  0.00285339]), 'sampling_rate': 16000}, 'transcription': 'Ich kriegte die Einberufung zum Arbeitsdienst.'}


In [8]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch

In [9]:
dataset_dict = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=4)

Map (num_proc=4): 100%|██████████| 331/331 [00:00<00:00, 434.74 examples/s]


In [10]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [11]:
model.generation_config.language = "Polish"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None


In [13]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [14]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [15]:
import evaluate

metric = evaluate.load("wer")

In [16]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [17]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-pl",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=False,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

/opt/miniconda3/envs/whisper-train/lib/python3.12/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

max_steps is given, it will override any value given in num_train_epochs


In [20]:
processor.save_pretrained(training_args.output_dir)

[]

In [21]:
trainer.train()

ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: [transcription, audio]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.